<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Greek/FastText_biGRU_crossval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Greek Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install focal-loss
!pip install keras-radam

  Created wheel for keras-radam: filename=keras_radam-0.15.0-cp36-none-any.whl size=14685 sha256=f3ffe981a84a33f55b7ac7dc96d74f5e2b3de91796bc76527bc8762be3675c8a
  Stored in directory: /root/.cache/pip/wheels/79/a0/c0/670b0a118e8f078539fafec7bd02eba0af921f745660c7f83f
Successfully built keras-radam


In [3]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Greek/

/content/drive/My Drive/dataset/OffenseEval2020/data/Greek


In [4]:
ls

offenseval-greek-training-v1.tsv  readme-trainingset-greek.txt


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-greek-training-v1.tsv',sep="\t")
train=train.sample(frac=1)

In [6]:
train.head()

,id,tweet,subtask_a
11,7287,Όταν δεν έχεις που να βάλεις το drone #gntmgr,NOT
2369,447,Είσαι τ' άγγιγμα του ανέμου που μου ξαφνιάζει ...,NOT
2061,7520,Τράβα κόψε κάνα εισιτήριο μωρή #TheVoiceGR,OFF
2160,7997,Ποιος ξέρει τι μαναφούκι θα γίνει με τη Μαρίνα...,NOT
6751,5415,"Παλι καλα που το παραδεχεται κιολας ,Κακο σκυλ...",OFF


In [7]:
train.tail()

,id,tweet,subtask_a
5626,7716,@USER Σπρωξανε γυναίκα κ μωρο από σκάλες οι φί...,NOT
4681,1502,@USER Εισαι εκλογικός ;,NOT
5197,5091,Γιατί αυτές οι ff νομίζουν όλες ότι είναι δυνα...,NOT
4664,3503,@USER Είσαι,NOT
3348,901,"@USER εισαι αγπημενο ακκαου,αλλα εδω δε θα σου...",NOT


In [0]:
labels=train['subtask_a']
train=train['tweet']
labels=pd.factorize(labels)[0]

In [9]:
import collections
collections.Counter(labels)

Counter({0: 6257, 1: 2486})

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [11]:
len(sequences_matrix),len(labels)

(8743, 8743)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import numpy as np
from sklearn.model_selection import StratifiedKFold
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [13]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [14]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.vec.gz

--2020-02-16 17:17:22--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1256281783 (1.2G) [binary/octet-stream]
Saving to: ‘cc.el.300.vec.gz’

cc.el.300.vec.gz    100%[===================>]   1.17G  21.0MB/s    in 48s     

2020-02-16 17:18:11 (24.9 MB/s) - ‘cc.el.300.vec.gz’ saved [1256281783/1256281783]



In [0]:
# import gzip
# import shutil
# with gzip.open('cc.el.300.vec.gz', 'rb') as f_in:
#     with open('cc.el.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embeddings=build_matrix('cc.el.300.vec', word_index)


In [18]:
embeddings.shape

(10001, 300)

In [0]:
def model_func():
  model = Sequential()
  model.add(Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False))
  model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
  #model.add(Bidirectional(CuDNNGRU(32, return_sequences = True)))
  model.add(GlobalMaxPool1D())
  model.add(Dense(50, activation="relu"))
  model.add(Dense(20, activation="relu"))
  model.add(Dense(10, activation="relu"))

  model.add(Dropout(0.05))
  model.add(Dense(1, activation="sigmoid"))
  model.compile(loss=BinaryFocalLoss(gamma=3), optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [22]:
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=2020)
cvscores = []

for train, test in kfold.split(sequences_matrix, labels):
  model=model_func()
  
  model.fit(sequences_matrix[train], labels[train],batch_size=256,epochs=10,verbose=2,class_weight=class_weights,
            validation_data=(sequences_matrix[test],labels[test]),callbacks=[reduce_lr])
  y_pred = model.predict(sequences_matrix[test], batch_size=128, verbose=1)
  y_pred_bool = np.argmax(y_pred, axis=1)
  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[test], y_pred, average='macro')
  cvscores.append(f1)
  keras.backend.clear_session()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train on 7868 samples, validate on 875 samples
Epoch 1/10
 - 17s - loss: 0.0755 - acc: 0.7143 - val_loss: 0.0717 - val_acc: 0.7154
Epoch 2/10
 - 2s - loss: 0.0693 - acc: 0.7157 - val_loss: 0.0667 - val_acc: 0.7154
Epoch 3/10
 - 2s - loss: 0.0644 - acc: 0.7173 - val_loss: 0.0635 - val_acc: 0.7371
Epoch 4/10
 - 2s - loss: 0.0600 - acc: 0.7869 - val_loss: 0.0624 - val_acc: 0.7817
Epoch 5/10
 - 1s - loss: 0.0575 - acc: 0.8129 - val_loss: 0.0618 - val_acc: 0.7874
Epoch 6/10
 - 2s - loss: 0.0551 - acc: 0.8263 - val_loss: 0.0639 - val_acc: 0.7966
Epoch 7/10
 - 2s - loss: 0.0519 - acc: 0.8474 - val_loss: 0.0649 - val_acc: 0.8103

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 8/10
 - 2s - loss: 0.0479 - acc: 0.8578 - val_loss: 0.0653 - val_acc: 0.8103
Epoch 9/10
 - 2s - loss: 0.0466 - acc: 0.8616 - val_loss: 0.0643 - val_acc: 0.8091

Epoch 00009: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 10/10
 - 2s - loss: 0.0453 - acc: 0

In [23]:
np.array(cvscores).mean()

0.7711031039511097